In [ ]:
import numpy as np
from typing import Tuple
import qiskit.quantum_info as qi
import cirq


In [5]:
X = qi.Pauli('X').to_matrix()
Y = qi.Pauli('Y').to_matrix()
Z = qi.Pauli('Z').to_matrix()
I = qi.Pauli('I').to_matrix()
XX = qi.Pauli('XX').to_matrix()
YY = qi.Pauli('YY').to_matrix()
ZZ = qi.Pauli('ZZ').to_matrix()

def canonical_decompose(u: np.ndarray, return_weyl_coord: bool = True) -> Tuple[
    Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray],
    Tuple[float, float, float]]:
    r"""
    Decompose a 4x4 unitary matrix into two pairs of single-qubit gates and three interaction coefficients.
    - If return_weyl_coord is True: returned coord is Weyl coordinate defined in
        (x, y, z) ~ e^{-i (x XX + y YY + z ZZ)} where (x, y, z) ∈ {π/4 ≥ x ≥ y ≥ |z| ≥ 0}
    - If return_weyl_coord is False: returned coord is KAK coefficients defined in
        (x, y, z) ~ e^{i (x XX + y YY + z ZZ)} where (x, y, z) ∈ {π/4 ≥ x ≥ y ≥ |z| ≥ 0}
    """
    res = cirq.kak_decomposition(u)
    coord = res.interaction_coefficients
    b1, b2 = res.single_qubit_operations_before
    a1, a2 = res.single_qubit_operations_after
    if return_weyl_coord:
        coord = (coord[0], coord[1], -coord[2])
        a1 = a1 @ Z
        b1 = Z @ b1
        if np.isclose(coord[0], np.pi / 4) and fuzzy_compare(coord[2], 0, '<'):
            coord = (coord[0], coord[1], -coord[2])
            a1 = a1 @ X
            a2 = a2 @ Z
            b2 = Y @ b2

    return (a1, a2), (b1, b2), coord

def fuzzy_compare(a, b, op, rtol=1e-7, atol=1e-10):
    """Comparison function with tolerance for floating point errors."""
    if op == ">=":
        return a > b or np.isclose(a, b, rtol=rtol, atol=atol)
    elif op == "<=":
        return a < b or np.isclose(a, b, rtol=rtol, atol=atol)
    elif op == ">":
        return a > b and not np.isclose(a, b, rtol=rtol, atol=atol)
    elif op == "<":
        return a < b and not np.isclose(a, b, rtol=rtol, atol=atol)
    elif op == "==":
        return np.isclose(a, b, rtol=rtol, atol=atol)
    elif op == "!=":
        return not np.isclose(a, b, rtol=rtol, atol=atol)
    else:
        raise ValueError("Unsupported operator (options: >=, <=, >, <, ==)")


In [9]:
CX = cirq.unitary(cirq.CNOT)
CZ = cirq.unitary(cirq.CZ)
SWAP = cirq.unitary(cirq.SWAP)

In [11]:
(a1, a2), (b1, b2), coord = canonical_decompose(CX)

In [15]:
a2.imag

array([[ 1.39427660e-16, -3.82683432e-01],
       [-3.82683432e-01,  1.39427660e-16]])

In [ ]:
cirq.two_qubit_matrix_to_sqrt_iswap_operations

In [ ]:
from qiskit.synthesis import TwoQubitBasisDecomposer

In [ ]:
from bqskit.compiler.compile import compile

In [10]:
canonical_decompose(SWAP)

((array([[0.00000000e+00+0.j, 5.55111512e-17-1.j],
         [5.55111512e-17-1.j, 0.00000000e+00+0.j]]),
  array([[-0.70710678+0.70710678j,  0.        +0.j        ],
         [ 0.        +0.j        ,  0.70710678+0.70710678j]])),
 (array([[ 0.70710678+0.70710678j,  0.        +0.j        ],
         [ 0.        +0.j        , -0.70710678+0.70710678j]]),
  array([[ 0.+0.00000000e+00j, -1.-5.55111512e-17j],
         [-1.-5.55111512e-17j,  0.+0.00000000e+00j]])),
 (np.float64(0.7853981633974483),
  np.float64(0.7853981633974483),
  np.float64(0.7853981633974483)))